In [1]:
import pandas as pd
from glob import glob
import numpy as np
from tqdm import tqdm

In [2]:
# groups = {}
# for group in tqdm(["CN","MCI","AD"],"Opening faster"):
#     groups[group]=np.load(open(f"Data/{group}.npy","br"))
#     print(len(groups[group]))

In [3]:
all_subjects = []
all_images = []
# patient_groups = {}
for group in ["CN","MCI","AD"]:
    files = glob(f"Data/Pre-processed/{group}/*.nii.gz")
    # with open(f"Data/{group}.npy","br") as f:
        # images = np.load(f)
    # assert len(files) == len(images) # Make sure we're not smoked
    # all_images.extend(images)
    # patient_groups[group] = patients, images
    'Data/Pre-processed/CN\\1000_sub-ADNI941S6546_ses-M048_T1w2_norm.nii.gz'
    def extract_sub(fp):
        sub = fp[fp.index("S"):]
        return sub[:sub[3:].index("_")+3].replace("_","")
    subjects = [extract_sub(fp) for fp in files]
    all_subjects.extend(subjects)

# all_images = np.stack(all_images)
# print(all_images.shape)

all_subjects = pd.DataFrame(all_subjects, columns=["subject"])
all_subjects.reset_index(inplace=True)
all_subjects.rename(columns={"index":"imageid"},inplace=True)

all_subjects["Group"] = 0
all_subjects.loc[1326:1326+461,"Group"] = 1
all_subjects.loc[1326+461:1326+461+213,"Group"] = 2

all_subjects

,imageid,subject,Group
0,0,S6546,0
1,1,S6570,0
2,2,S6570,0
3,3,S6574,0
4,4,S6574,0
...,...,...,...
1995,1995,S0300,2
1996,1996,S0916,2
1997,1997,S1262,2
1998,1998,S0404,2


In [4]:
def open_prepare_df(name="All_Subjects_RECCMEDS_11Mar2025"):
    try:
        df = pd.read_csv(f"Data/Individual level/{name}.csv")
    except Exception as e:
        if "Archived" in name: raise e 
        return open_prepare_df("Archived/"+name)
    df["subject"] = df["PTID"].str.split("_")
    df["subject"] = df["subject"].str[1] + df["subject"].str[2]
    return df

adverse_events = open_prepare_df("All_Subjects_ADVERSE_11Mar2025")

A=set(adverse_events["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))
adv_events = adverse_events[["subject","AERELAD","AERELCM","AERELFLRBTBN","AERELFLRBPR","AEHIMG","AERELTAU","AERELNAV","AERELMK","AERELPI","AEHLUMB","AERELCOVID","AERELPAN","AERELATESP","AEHCMEDS","AESERIOUS"]].fillna(0)
for col in adv_events:
    if col =="subject": continue
    col_ = adv_events[col]
    adv_events[col] = col_.astype("float")
    adv_events.loc[col_ > 0, col] = (5 - col_) / 4.
adv_events

1120 378 904 594


,subject,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,AERELNAV,AERELMK,AERELPI,AEHLUMB,AERELCOVID,AERELPAN,AERELATESP,AEHCMEDS,AESERIOUS
0,S2245,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,1.00,0.0,0.0,0.00,1.0,0.0
1,S6041,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.50,0.0,0.0,0.00,0.0,0.0
2,S6053,0.0,0.0,0.0,0.25,0.25,0.25,0.0,0.0,0.0,0.75,0.0,0.0,0.25,1.0,0.0
3,S6005,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
4,S2239,0.0,0.5,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3420,S7011,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
3421,S10031,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,1.0,0.0
3422,S10031,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.0
3423,S6288,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.00,1.0,0.0


In [5]:
num_events = all_subjects.groupby(by="subject").size().reset_index().rename(columns={0:"num_events"})
all_subjects = all_subjects.merge(num_events,how="left").fillna(0)
all_subjects = all_subjects.merge(adv_events.groupby(by="subject").sum().reset_index(),how="left").fillna(0)
all_subjects

,imageid,subject,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,AERELNAV,AERELMK,AERELPI,AEHLUMB,AERELCOVID,AERELPAN,AERELATESP,AEHCMEDS,AESERIOUS
0,0,S6546,0,4,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0
1,1,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
4,4,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2,12,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,1996,S0916,2,18,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,1997,S1262,2,14,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,1998,S0404,2,11,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
backmeds = open_prepare_df("All_Subjects_BACKMEDS_11Mar2025")
backmeds = backmeds[["subject","KEYMED"]]
med_vocab = list(map(str,list(range(8))))
backmeds_ = []
names = ["No med",
"Aricept",
"Cognex",
"Exelon",
"Namenda",
"Razadyne",
"Anti-depressant",
"Other med"]
remap = {str(i):names[i] for i in range(len(names))}
for i, row in backmeds.iterrows():
    data = {"subject":row["subject"]}
    for word in med_vocab:
        data[remap[word]] = int(word in str(row["KEYMED"]))
    backmeds_.append(data)
backmeds_=pd.DataFrame(backmeds_)
# backmeds_["Num_meds"] = backmeds_.drop(columns=["subject","No med"]).sum(axis=1)
backmeds_["Num_visits"] = 1
backmeds_ = backmeds_.groupby(by="subject").sum().reset_index()
for col in backmeds_:
    if col in ["subject", "Num_visits"]: continue
    backmeds_[col] /= backmeds_["Num_visits"]

all_subjects=all_subjects.merge(backmeds_.groupby(by="subject").sum().reset_index(),how="left").fillna(0)
all_subjects.loc[all_subjects["Num_visits"]==0, "No med"] = 1
all_subjects

,imageid,subject,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,...,AESERIOUS,No med,Aricept,Cognex,Exelon,Namenda,Razadyne,Anti-depressant,Other med,Num_visits
0,0,S6546,0,4,0.0,0.25,0.0,0.0,0.0,0.0,...,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1,1,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
2,2,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,3,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,4,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2,12,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1996,1996,S0916,2,18,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1997,1997,S1262,2,14,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1998,1998,S0404,2,11,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
neuropath = open_prepare_df("All_Subjects_NEUROPATH_11Mar2025")
neuropath = neuropath[["subject","NPWBRWT","NPGRCCA","NPGRLA","NPGRHA","NPGRSNH","NPGRLCH","NPAVAS","NPTAN","NPABAN","NPASAN","NPTDPAN","NPHISMB","NPHISG","NPHISSS","NPHIST","NPHISO","NPTHAL","NPBRAAK","NPNEUR","NPADNC","NPDIFF","NPAMY","NPINF","NPHEMO","NPOLD","NPARTER","NPWMR","NPPATH","NPNEC","NPPATHO","NPLBOD","NPNLOSS","NPHIPSCL","NPTDPA","NPTDPB","NPTDPC","NPTDPD","NPTDPE","NPFTDTAU","NPPICK","NPPDXS"]]

A=set(neuropath["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))

# Due to the very limited amount of subjects who have been autopsied in our imaging data, we dont need to merge. not enough data.
# all_subjects=all_subjects.merge(neuropath,how="left")
# all_subjects

692 12 110 594


In [8]:
reccmed = pd.read_csv("Data/Individual level/Archived/All_Subjects_RECCMEDS_11Mar2025.csv", dtype="str")
reccmed["subject"] = reccmed["PTID"].str.split("_")
reccmed["subject"] = reccmed["subject"].str[1]+reccmed["subject"].str[2]

A=set(reccmed["subject"])
B=set(all_subjects["subject"])
# print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))

reccmed = reccmed[["subject","CMMED","CMFREQNC","CMREASON", "CMBGN","CMEND","update_stamp"]]
reccmed["CMMED"] = reccmed["CMMED"].str.lower().str.replace("-"," ")
for suffix in [" formula", " oil"," sltb"," la"]:
    reccmed["CMMED"] = reccmed["CMMED"].str.removesuffix(suffix)
for overwrite in ["aspirin","oxybutynin","calcium","garlic","omega 3","probiotic","lactase","acetyl","vitamin b","vitamin c","ascorbic acid",
                  "tylenol","glucosamine","multivitamin","triamter","elidel","nephrovite","ammonium lactate","acetaminophen"]:
    mask = reccmed["CMMED"].str.contains(overwrite).fillna(0).astype("bool")
    reccmed.loc[mask, "CMMED"] = overwrite
# reccmed = reccmed[~pd.isna(reccmed["CMMED"])]
vocab = set(reccmed["CMMED"])
print(len(vocab))
vocab

3442


{'xalantan',
 'memory essentials',
 'fluticasone',
 'robitussin',
 'mementine',
 'adalat',
 'magnesium 64 supplement',
 'liothyronine',
 'glyburide/metformin',
 'salbuterol',
 'neurontin',
 'ginko biloba plus tablet',
 'hydrocodone bitartrate',
 'ciprofloxacin',
 'estriol',
 'tonic water',
 'synthyroid',
 'sameterol disc inhaler',
 'gentamicin sulfate 0.3%',
 'similimilasan ear drops',
 'vit eyes',
 'linisopril',
 'omacor',
 'gravol',
 'cosopt[|194|][|174|] (dorzolamide hydrochloride timolol maleate) sterile ophthalmic solution',
 'oxtrol',
 'muitivitamin',
 'proflavenol',
 'uromax',
 'ceriva',
 'musinex',
 'actenol',
 'triamcinolone',
 'nortriptyline',
 'beconase',
 'diflunisal',
 'formotoral fumarate',
 'phoshatodylserine',
 'topomax',
 'testosterone gel',
 'xylocaine',
 'triamcinolone acetonide',
 'flonaz',
 'ayr saline nasal spray',
 'calcitriol',
 'emo cort cream 2.5%',
 'exforgen',
 'fennel',
 'kenalog',
 'theopylline',
 'glucovance',
 'duo neb',
 'estrace cream',
 'pms methyl ph

In [9]:
common_meds_vocab = reccmed["CMMED"].value_counts().head(30).keys().values

one_hot_df = pd.get_dummies(reccmed["CMMED"])[common_meds_vocab].astype("int")
one_hot_df.columns = ["med_"+col for col in one_hot_df.columns]
one_hot_df.rename(columns={"med_ 4":"med_unknown"},inplace=True)

reccmed_ = pd.concat([reccmed[["subject","CMREASON","CMFREQNC"]], one_hot_df], axis=1)
reccmed_

,subject,CMREASON,CMFREQNC,med_unknown,med_aspirin,med_multivitamin,med_calcium,med_aricept,med_namenda,med_fish,...,med_hydrochlorothiazide,med_omeprazole,med_folic acid,med_synthroid,med_atenolol,med_metoprolol,med_omega 3,med_metformin,med_prednisone,med_flomax
0,S0002,NaN,NaN,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,S0003,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,S0003,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,S0003,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,S0003,NaN,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72563,S10586,3,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72564,S10586,3,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72565,S10586,2,13,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
72566,S10586,2,10,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
groups = reccmed_.drop(columns=["CMFREQNC","CMREASON"]).groupby(by="subject")
all_meds_hist = groups.mean()
all_meds_hist

,med_unknown,med_aspirin,med_multivitamin,med_calcium,med_aricept,med_namenda,med_fish,med_vitamin b,med_vitamin c,med_vitamin d,...,med_hydrochlorothiazide,med_omeprazole,med_folic acid,med_synthroid,med_atenolol,med_metoprolol,med_omega 3,med_metformin,med_prednisone,med_flomax
subject,,,,,,,,,,,,,,,,,,,,,
S0002,0.300000,0.100000,0.033333,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.066667,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
S0003,0.111111,0.055556,0.055556,0.000000,0.000000,0.277778,0.000000,0.0,0.055556,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
S0004,0.043478,0.000000,0.000000,0.000000,0.043478,0.000000,0.043478,0.0,0.043478,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
S0005,0.095238,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.047619,0.0,0.0,0.0
S0006,0.210526,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.052632,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S7122,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
S7123,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0
S7124,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0


In [11]:
all_subjects = all_subjects.merge(all_meds_hist.reset_index(),how="left")
all_subjects

,imageid,subject,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,...,med_hydrochlorothiazide,med_omeprazole,med_folic acid,med_synthroid,med_atenolol,med_metoprolol,med_omega 3,med_metformin,med_prednisone,med_flomax
0,0,S6546,0,4,0.0,0.25,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
1,1,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
2,2,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
3,3,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
4,4,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2,12,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0
1996,1996,S0916,2,18,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.04,0.0,0.04,0.0,0.000000,0.0,0.0,0.0
1997,1997,S1262,2,14,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.076923,0.0,0.0,0.0
1998,1998,S0404,2,11,0.0,0.00,0.0,0.0,0.0,0.0,...,0.0,0.0,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0


In [12]:
# FAMHXSIB_21Mar2025 FAMHXPAR_21Mar2025 PHYSICAL_21Mar2025 PTDEMOG_21Mar2025 VITALS_21Mar2025
siblings = open_prepare_df("FAMHXSIB_21Mar2025")
A=set(siblings["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))
siblings = siblings[["subject","SIBDEMENT"]]
siblings["Num_siblings"] = 1
siblings = siblings.groupby(by="subject").sum()
siblings["SIBDEMENT"] /= siblings["Num_siblings"]
all_subjects = all_subjects.merge(siblings,on="subject",how="left")
naive_dementia_rate = 0.001 # siblings["SIBDEMENT"].mean()
all_subjects.loc[pd.isna(all_subjects["SIBDEMENT"]), "SIBDEMENT"] = naive_dementia_rate
all_subjects = all_subjects.fillna(siblings["Num_siblings"].median()) # Num_siblings unknown set to zero
all_subjects

1836 402 1644 594


,imageid,subject,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,...,med_folic acid,med_synthroid,med_atenolol,med_metoprolol,med_omega 3,med_metformin,med_prednisone,med_flomax,SIBDEMENT,Num_siblings
0,0,S6546,0,4,0.0,0.25,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.001,3.0
1,1,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.000,2.0
2,2,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.000,2.0
3,3,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.000,2.0
4,4,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.000,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2,12,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.001,3.0
1996,1996,S0916,2,18,0.0,0.00,0.0,0.0,0.0,0.0,...,0.04,0.0,0.04,0.0,0.000000,0.0,0.0,0.0,0.001,3.0
1997,1997,S1262,2,14,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.076923,0.0,0.0,0.0,0.001,3.0
1998,1998,S0404,2,11,0.0,0.00,0.0,0.0,0.0,0.0,...,0.00,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.001,3.0


In [13]:
# FAMHXSIB_21Mar2025 FAMHXPAR_21Mar2025 PHYSICAL_21Mar2025 PTDEMOG_21Mar2025 VITALS_21Mar2025
siblings = open_prepare_df("FAMHXPAR_21Mar2025")
A=set(siblings["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))
siblings = siblings[["subject","MOTHDEM","FATHDEM","MOTHAGE","FATHAGE"]]
siblings = siblings.groupby(by="subject").mean()
all_subjects = all_subjects.merge(siblings,on="subject",how="left")
all_subjects["MOTHDEM"] = all_subjects["MOTHDEM"].fillna(siblings["MOTHDEM"].mean())
all_subjects["FATHDEM"] = all_subjects["FATHDEM"].fillna(siblings["FATHDEM"].mean())
all_subjects

1914 421 1741 594


,imageid,subject,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,...,med_omega 3,med_metformin,med_prednisone,med_flomax,SIBDEMENT,Num_siblings,MOTHDEM,FATHDEM,MOTHAGE,FATHAGE
0,0,S6546,0,4,0.0,0.25,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.001,3.0,1.00000,0.000000,94.0,69.0
1,1,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000,2.0,0.00000,1.000000,81.0,82.0
2,2,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000,2.0,0.00000,1.000000,81.0,82.0
3,3,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000,2.0,1.00000,0.000000,94.0,52.0
4,4,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.000,2.0,1.00000,0.000000,94.0,52.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2,12,0.0,0.00,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.001,3.0,0.49157,0.284337,NaN,NaN
1996,1996,S0916,2,18,0.0,0.00,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.001,3.0,0.49157,0.284337,NaN,NaN
1997,1997,S1262,2,14,0.0,0.00,0.0,0.0,0.0,0.0,...,0.076923,0.0,0.0,0.0,0.001,3.0,0.49157,0.284337,NaN,NaN
1998,1998,S0404,2,11,0.0,0.00,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.001,3.0,0.49157,0.284337,NaN,NaN


In [14]:
# FAMHXSIB_21Mar2025 FAMHXPAR_21Mar2025 PHYSICAL_21Mar2025 PTDEMOG_21Mar2025 VITALS_21Mar2025
siblings = open_prepare_df("PHYSICAL_21Mar2025")
A=set(siblings["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))
siblings = siblings[["subject"] + [c for c in siblings.columns if c.startswith("PX")]]
siblings = siblings.groupby(by="subject").mean()

all_subjects = all_subjects.merge(siblings,on="subject",how="left")
# all_subjects = all_subjects.fillna(1) # In case we have missing values, 1 represents a "normal" case.
all_subjects

4043 594 4043 594


,imageid,subject,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,...,PXHEART,PXABDOM,PXEXTREM,PXEDEMA,PXPERIPH,PXSKIN,PXMUSCUL,PXBACK,PXOTHER,PXABNORM
0,0,S6546,0,4,0.0,0.25,0.0,0.0,0.0,0.0,...,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,NaN,1.0
1,1,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,1.5,1.0,1.0,1.0,1.0,1.5,1.5,1.0,1.0,1.0
2,2,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,1.5,1.0,1.0,1.0,1.0,1.5,1.5,1.0,1.0,1.0
3,3,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,2.0,1.5,2.0,1.5,1.0,1.0,1.0
4,4,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,2.0,2.0,1.5,2.0,1.5,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2,12,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0
1996,1996,S0916,2,18,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,2.0,2.0,1.0,NaN,1.0,1.0
1997,1997,S1262,2,14,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0
1998,1998,S0404,2,11,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,2.0,1.0,2.0,2.0,NaN,1.0,1.0


In [15]:
# FAMHXSIB_21Mar2025 FAMHXPAR_21Mar2025 PHYSICAL_21Mar2025 PTDEMOG_21Mar2025 VITALS_21Mar2025
siblings = open_prepare_df("PTDEMOG_21Mar2025")
A=set(siblings["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))
exclude = ["PTID","PTDOB","PTASIAN","PTADBEG","PTIDENT","PTORIENT","PTENGSPK","PTETHCATH","PTWORK","PTWORKHS","PTOPI","PTBIRPL","PTIMMAGE","PTIMMWHY","PTORIENTOT","PTENGSPKAGE","PTSPTIM","PTSPOTTIM","PTBORN","PTBIRPR","PTBIRGR"]
siblings = siblings[["subject"] + [c for c in siblings.columns if c.startswith("PT")and c not in exclude and "LANG" not in c]]
# siblings = siblings.astype("float")
siblings["PTRACCAT"] = siblings["PTRACCAT"].str.split("|").str[0].astype("float")
siblings = siblings.groupby(by="subject").mean()
# siblings
all_subjects = all_subjects.merge(siblings,on="subject",how="left")
all_subjects = all_subjects.fillna(1) # In case we have missing values, 1 represents a "normal" case.
all_subjects

4668 594 4668 594


,imageid,subject,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,...,PTHAND,PTMARRY,PTEDUCAT,PTNOTRT,PTRTYR,PTHOME,PTCOGBEG,PTADDX,PTETHCAT,PTRACCAT
0,0,S6546,0,4,0.0,0.25,0.0,0.0,0.0,0.0,...,1.0,2.0,20.0,1.0,2005.0,5.5,9999.0,9999.0,2.0,5.0
1,1,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,18.0,0.0,1.0,1.0,9999.0,9999.0,2.0,5.0
2,2,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,18.0,0.0,1.0,1.0,9999.0,9999.0,2.0,5.0
3,3,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,16.0,1.0,2007.0,5.5,9999.0,9999.0,2.0,5.0
4,4,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,16.0,1.0,2007.0,5.5,9999.0,9999.0,2.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2,12,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,16.0,1.0,2005.0,1.0,1.0,1.0,2.0,5.0
1996,1996,S0916,2,18,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,16.0,1.0,1984.0,2.0,1.0,1.0,2.0,5.0
1997,1997,S1262,2,14,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,3.0,12.0,1.0,2005.0,1.0,1.0,1.0,2.0,5.0
1998,1998,S0404,2,11,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,1.0,14.0,0.0,1.0,6.0,1.0,1.0,2.0,5.0


In [25]:
# FAMHXSIB_21Mar2025 FAMHXPAR_21Mar2025 PHYSICAL_21Mar2025 PTDEMOG_21Mar2025 VITALS_21Mar2025
siblings = open_prepare_df("VITALS_21Mar2025")
A=set(siblings["subject"])
B=set(all_subjects["subject"])
print(len(A.union(B)), len(A.intersection(B)), len(A), len(B))
exclude = ["VSHGTSC","VSTMPSRC"]
siblings = siblings[["subject"] + [c for c in siblings.columns if c.startswith("VS")and c not in exclude]]
siblings.loc[siblings["VSWTUNIT"]==1, "VSWEIGHT"] *= 0.45359237 # pounds
siblings.loc[siblings["VSHTUNIT"]==1, "VSHEIGHT"] *= 2.54 # inches
siblings.loc[siblings["VSTMPUNT"]==1, "VSTEMP"] = (siblings.loc[siblings["VSTMPUNT"]==1, "VSTEMP"] - 32) / 1.8 # fahrenheight
siblings = siblings.drop(columns=[c for c in siblings.columns if "UN" in c])
siblings = siblings.groupby(by="subject").mean()
# siblings
all_subjects = all_subjects.merge(siblings,on="subject",how="left")
all_subjects

4140 594 4140 594


,imageid,subject,Group,num_events,AERELAD,AERELCM,AERELFLRBTBN,AERELFLRBPR,AEHIMG,AERELTAU,...,PTADDX,PTETHCAT,PTRACCAT,VSWEIGHT,VSHEIGHT,VSBPSYS,VSBPDIA,VSPULSE,VSRESP,VSTEMP
0,0,S6546,0,4,0.0,0.25,0.0,0.0,0.0,0.0,...,9999.0,2.0,5.0,76.611751,167.000000,137.750000,70.250000,68.75,18.250000,36.819444
1,1,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,9999.0,2.0,5.0,90.869671,172.000000,149.000000,69.666667,58.00,18.666667,25.062963
2,2,S6570,0,3,0.0,0.00,0.0,0.0,0.0,0.0,...,9999.0,2.0,5.0,90.869671,172.000000,149.000000,69.666667,58.00,18.666667,25.062963
3,3,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,9999.0,2.0,5.0,58.096111,158.000000,114.200000,64.800000,70.40,17.600000,29.855556
4,4,S6574,0,5,0.0,0.00,0.0,0.0,0.0,0.0,...,9999.0,2.0,5.0,58.096111,158.000000,114.200000,64.800000,70.40,17.600000,29.855556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1995,S0300,2,12,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,2.0,5.0,84.096025,32.360000,135.400000,84.400000,66.80,17.200000,36.233333
1996,1996,S0916,2,18,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,2.0,5.0,71.050709,31.852000,158.600000,75.600000,51.80,19.600000,35.566667
1997,1997,S1262,2,14,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,2.0,5.0,50.711627,23.336667,135.666667,67.500000,69.00,17.666667,36.333333
1998,1998,S0404,2,11,0.0,0.00,0.0,0.0,0.0,0.0,...,1.0,2.0,5.0,65.625744,29.566000,127.800000,67.800000,68.40,19.200000,36.120000


In [26]:
all_subjects.to_csv("Data/subjects.csv")
# all_subjects = pd.read_csv("Data/subjects.csv").drop(columns="Unnamed: 0")
# all_subjects